<a href="https://colab.research.google.com/github/rjsdn0124/Practice_AI_Module/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 파이토치로 다중 로지스틱 회귀 구현하기

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

# 0.5가 넘으면 True르 반환시키고자 함.

x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

print(x_train.shape)
print(y_train.shape)

# X가 현재 [6,2] 형태이므로 [6,1]로 만들어주기 위해 WX형태로 만들어 주려면 W는 [2,1]이 되어야 해서 해당 형태로 설정.
W = torch.zeros((2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# W와 b는 현재 0으로 초기화된 상태. 이 상태에서 여러 과정을 통해 오차를 구해서 손실함수로 계산 후 기울기 재 설정 후 W,b 재설정.

# hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b))) 
# 위의 식을 torch에선 sigmoid 함수로 사용.
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis)
print(y_train)

# losses = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis))
# cost = losses.mean()
# 해당 식도 함수로 구현되어있음
cost = F.binary_cross_entropy(hypothesis, y_train)

# 훈련 과정>>
optimizer = optim.SGD([W, b], lr=1)

print('='*30)
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = -(y_train * torch.log(hypothesis) + 
             (1 - y_train) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))
        
print(hypothesis)
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction)

torch.Size([6, 2])
torch.Size([6, 1])
tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])
Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852
tensor([[2.7711e-04],
        [3.1636e-02],
        [3.9014e-02],
        [9.5618e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward0>)
tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])


## nn.Module을 통해 구현하기


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

model = nn.Sequential(
    nn.Linear(2,1),
    nn.Sigmoid()
)
# forward시킬 모듈 생성.

optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 2000
for epoch in range(nb_epochs + 1):

    # H(x) 계산 forward
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train) # 모델의 매개변수를 통해 해당 H(x)계산한 값이랑 오차를 계산해서 저장.

    # cost로 H(x) 개선
    optimizer.zero_grad() # 모델의 매개변수의 변화도를 재설정하기위해 0으로 초기화. 변화도는 계속 더해지는 코드라 0으로 계속 초기화 필요.
    cost.backward() # 모델의 매개변수의 변화도를 계산하여 재설정해야하는 변화량 계산.
    optimizer.step() # backward를 통해 model의 매개변수 객체에서 변화량이 변경된거를 확인하여 변경시켜주는 함수. 여기서 실질적 변경 일어남.

    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/2000 Cost: 0.539713 Accuracy 83.33%
Epoch   10/2000 Cost: 0.614853 Accuracy 66.67%
Epoch   20/2000 Cost: 0.441875 Accuracy 66.67%
Epoch   30/2000 Cost: 0.373145 Accuracy 83.33%
Epoch   40/2000 Cost: 0.316358 Accuracy 83.33%
Epoch   50/2000 Cost: 0.266094 Accuracy 83.33%
Epoch   60/2000 Cost: 0.220498 Accuracy 100.00%
Epoch   70/2000 Cost: 0.182095 Accuracy 100.00%
Epoch   80/2000 Cost: 0.157299 Accuracy 100.00%
Epoch   90/2000 Cost: 0.144091 Accuracy 100.00%
Epoch  100/2000 Cost: 0.134272 Accuracy 100.00%
Epoch  110/2000 Cost: 0.125769 Accuracy 100.00%
Epoch  120/2000 Cost: 0.118297 Accuracy 100.00%
Epoch  130/2000 Cost: 0.111680 Accuracy 100.00%
Epoch  140/2000 Cost: 0.105779 Accuracy 100.00%
Epoch  150/2000 Cost: 0.100483 Accuracy 100.00%
Epoch  160/2000 Cost: 0.095704 Accuracy 100.00%
Epoch  170/2000 Cost: 0.091369 Accuracy 100.00%
Epoch  180/2000 Cost: 0.087420 Accuracy 100.00%
Epoch  190/2000 Cost: 0.083806 Accuracy 100.00%
Epoch  200/2000 Cost: 0.080486 Accuracy 100.00

##. by Model


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

class BinaryClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(2,1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    return self.sigmoid(self.linear(x))

model = BinaryClassifier()

optimizer = optim.SGD(model.parameters(), lr = 1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))


Epoch    0/1000 Cost: 0.614994 Accuracy 66.67%
Epoch   10/1000 Cost: 0.747550 Accuracy 83.33%
Epoch   20/1000 Cost: 0.633216 Accuracy 83.33%
Epoch   30/1000 Cost: 0.538123 Accuracy 83.33%
Epoch   40/1000 Cost: 0.450406 Accuracy 83.33%
Epoch   50/1000 Cost: 0.366382 Accuracy 83.33%
Epoch   60/1000 Cost: 0.287368 Accuracy 83.33%
Epoch   70/1000 Cost: 0.219289 Accuracy 83.33%
Epoch   80/1000 Cost: 0.173225 Accuracy 100.00%
Epoch   90/1000 Cost: 0.151674 Accuracy 100.00%
Epoch  100/1000 Cost: 0.140280 Accuracy 100.00%
Epoch  110/1000 Cost: 0.131002 Accuracy 100.00%
Epoch  120/1000 Cost: 0.122903 Accuracy 100.00%
Epoch  130/1000 Cost: 0.115765 Accuracy 100.00%
Epoch  140/1000 Cost: 0.109426 Accuracy 100.00%
Epoch  150/1000 Cost: 0.103760 Accuracy 100.00%
Epoch  160/1000 Cost: 0.098664 Accuracy 100.00%
Epoch  170/1000 Cost: 0.094056 Accuracy 100.00%
Epoch  180/1000 Cost: 0.089870 Accuracy 100.00%
Epoch  190/1000 Cost: 0.086050 Accuracy 100.00%
Epoch  200/1000 Cost: 0.082549 Accuracy 100.00%
